## NIPS 2017 Figure 4: Bad data and missing features

In [5]:
%matplotlib inline
from __future__ import division
import numpy as np
import scipy.signal as ss
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from math import factorial

import likelihoodfree.io as io
import likelihoodfree.viz as viz
import likelihoodfree.PDF as lfpdf
import os

import svgutils.transform as sg

def cm2px(cm):
    return int(np.round(cm / 2.54*300))

In [9]:
#create new SVG figure
fig = sg.SVGFigure("16cm", "6.5cm")

# load matpotlib-generated figures
dir_plots = '../results/'
prefix = ''

#fig1 = sg.fromfile(dir_plots + 'placeholder' + prefix + '.svg')
#plot1 = fig1.getroot()
#plot1.moveto(15, 15, scale=1.)
#fig.append([plot1])

# add text labels
rows = 2

r1width = cm2px(16)
r1cols  = 3
r1wpc   = int(np.floor(r1width / r1cols))

r2width = cm2px(16)
r2cols  = 3
r2wpc   = int(np.floor(r2width / r2cols))

xlspc   = 10
ylspc   = 15

rheight = int(cm2px(6.5)/rows)

xtspc   = 100
ytspc   = 150

sletter = 20
stext   = 18

fig.append(sg.TextElement(0*r1wpc + xlspc, 0*rheight + ylspc,  "A", size=sletter, weight="bold"))
fig.append(sg.TextElement(0*r1wpc + xtspc, 0*rheight + ytspc,  "Problem: Simulator blow up", stext, weight="bold"))

fig.append(sg.TextElement(1*r1wpc + xlspc, 0*rheight + ylspc,  "B", size=sletter, weight="bold"))
fig.append(sg.TextElement(1*r1wpc + xtspc, 0*rheight + ytspc,  "Solution: Train auxillary model g_hat", stext, weight="bold"))

fig.append(sg.TextElement(2*r1wpc + xlspc, 0*rheight + ylspc,  "C", size=sletter, weight="bold"))
fig.append(sg.TextElement(2*r1wpc + xtspc, 0*rheight + ytspc,  "Results: Bad data handling scheme/not", stext, weight="bold"))
fig.append(sg.TextElement(2*r1wpc + xtspc, 0*rheight + ytspc + 20, "... autapse (ground truth known) and HH", stext, weight="bold"))

fig.append(sg.TextElement(0*r2wpc + xlspc, 1*rheight + ylspc,  "D", size=sletter, weight="bold"))
fig.append(sg.TextElement(0*r2wpc + xtspc, 1*rheight + ytspc,  "Problem: Missing features; HH traces", stext, weight="bold"))

fig.append(sg.TextElement(1*r2wpc + xlspc, 1*rheight + ylspc,  "E", size=sletter, weight="bold"))
fig.append(sg.TextElement(1*r2wpc + xtspc, 1*rheight + ytspc,  "Solution: Scheme of NN for imputing missing values", stext, weight="bold"))

fig.append(sg.TextElement(2*r2wpc + xlspc, 1*rheight + ylspc,  "F", size=sletter, weight="bold"))
fig.append(sg.TextElement(2*r2wpc + xtspc, 1*rheight + ytspc,  "Results: Missing value imputation results", stext, weight="bold"))

# quick fix
fig.append(sg.TextElement(0, 0,  ".", size=1, weight="bold"))
fig.append(sg.TextElement(cm2px(16), cm2px(6.5),  ".", size=1, weight="bold"))

# save figure
# https://tex.stackexchange.com/questions/2099/how-to-include-svg-diagrams-in-latex
fig.save(dir_plots + 'NIPS2017_fig4.svg')

In [10]:
!inkscape -D -z --export-pdf ~/repos/likelihoodfree-models/results/fig4.pdf ~/repos/likelihoodfree-models/results/NIPS2017_fig4.svg